# Extracción de datos desde APIs con Python


## Conceptos básicos de APIs REST y consumo de APIs en Python

### ¿Qué son las API?

- Son un conjuntos de reglas y protocolos que permiten a diferentes aplicaciones comunicarse entre sí. 
- Se utilizan para acceder a datos de servicios web, aplicaciones en la nube, redes sociales, entre otros. 
- Proporcionan una forma estructurada y segura de obtener datos de sistemas externos.

![API](../Images/API.png)

## Extracción de datos JSON desde APIs públicas y privadas

![API_Spotify](../Images/API_Spotify.png)

In [1]:
import pandas as pd
import os
import json
import requests
import re
import base64
from requests.auth import HTTPBasicAuth

In [2]:
path = os.getcwd()
path_credenciales = os.path.join(path, 'secretos', 'credentials_spoty.json')
with open(path_credenciales) as f:
    credenciales = json.load(f)
    client_id = credenciales['client_id']
    client_secret = credenciales['client_secret']

data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

token_url = 'https://accounts.spotify.com/api/token'
response = requests.post(token_url, data=data)

if response.status_code == 200:
    token = response.json()['access_token']
    print('Token válido')
else:
    print('Error en la obtención del token')
    print(response.text)

Token válido


In [3]:
# Usando el mismo token que ya tienes
url = 'https://api.spotify.com/v1/search'
params = {
    'q': 'The Beatles',  # Término de búsqueda
    'type': 'track',     # Buscamos tracks
    'limit': 5           # Limitamos a 5 resultados
}
headers = {
    'Authorization': f'Bearer {token}'
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

# Para ver los resultados de manera más ordenada
if response.status_code == 200:
    tracks = data['tracks']['items']
    for track in tracks:
        print(f"Nombre: {track['name']}")
        print(f"Artista: {track['artists'][0]['name']}")
        print(f"Album: {track['album']['name']}")
        print("-------------------")

Nombre: And I Love Her - Remastered 2009
Artista: The Beatles
Album: A Hard Day's Night (Remastered)
-------------------
Nombre: Here Comes The Sun - Remastered 2009
Artista: The Beatles
Album: Abbey Road (Remastered)
-------------------
Nombre: Now And Then
Artista: The Beatles
Album: Now And Then
-------------------
Nombre: The Beatles “Julia” - Franco Piccinno, piano
Artista: Franco Piccinno
Album: The Beatles “Julia” - Franco Piccinno, piano
-------------------
Nombre: Strawberry Fields Forever - Remastered 2009
Artista: The Beatles
Album: Magical Mystery Tour (Remastered)
-------------------


Consumo de la API de [mockaroo](https://www.mockaroo.com/), que es una herramienta que permite generar datos de prueba en diferentes formatos, como JSON, CSV, SQL y Excel.

In [4]:
import requests
headers = {'X-Api-Key': '679636c0'}
response = requests.get('https://my.api.mockaroo.com/prueba.json', headers=headers)

In [5]:
response.json()

[{'id': 1,
  'first_name': 'Abagael',
  'last_name': 'Caesmans',
  'email': 'acaesmans0@aol.com',
  'gender': 'Female',
  'ip_address': '247.87.95.240'},
 {'id': 2,
  'first_name': 'Mitch',
  'last_name': 'Ortmann',
  'email': 'mortmann1@patch.com',
  'gender': 'Male',
  'ip_address': '49.183.66.196'},
 {'id': 3,
  'first_name': 'Cammi',
  'last_name': 'Poncet',
  'email': 'cponcet2@alibaba.com',
  'gender': 'Female',
  'ip_address': '252.58.137.136'},
 {'id': 4,
  'first_name': 'Ludovico',
  'last_name': 'Eaklee',
  'email': 'leaklee3@guardian.co.uk',
  'gender': 'Male',
  'ip_address': '6.205.132.155'},
 {'id': 5,
  'first_name': 'Tanney',
  'last_name': 'Roncelli',
  'email': 'troncelli4@51.la',
  'gender': 'Male',
  'ip_address': '68.33.10.64'},
 {'id': 6,
  'first_name': 'Maurise',
  'last_name': "D'Agostino",
  'email': 'mdagostino5@abc.net.au',
  'gender': 'Female',
  'ip_address': '194.55.204.135'},
 {'id': 7,
  'first_name': 'Isadora',
  'last_name': 'Gillcrist',
  'email': 'i

### Pandas JSON

In [6]:
import pandas as pd
from io import StringIO

In [7]:
pd.read_json(StringIO(response.text))

,id,first_name,last_name,email,gender,ip_address
0,1,Abagael,Caesmans,acaesmans0@aol.com,Female,247.87.95.240
1,2,Mitch,Ortmann,mortmann1@patch.com,Male,49.183.66.196
2,3,Cammi,Poncet,cponcet2@alibaba.com,Female,252.58.137.136
3,4,Ludovico,Eaklee,leaklee3@guardian.co.uk,Male,6.205.132.155
4,5,Tanney,Roncelli,troncelli4@51.la,Male,68.33.10.64
5,6,Maurise,D'Agostino,mdagostino5@abc.net.au,Female,194.55.204.135
6,7,Isadora,Gillcrist,igillcrist6@alexa.com,Polygender,240.218.253.208
7,8,Ricard,Rykert,rrykert7@geocities.com,Male,85.80.86.220
8,9,Damian,Whitehair,dwhitehair8@technorati.com,Male,213.156.182.21
9,10,Timmy,Berney,tberney9@vk.com,Male,231.206.159.223


In [8]:
path_data = os.path.join(os.getcwd(), 'data')

ejemplo1 = os.path.join(path_data, 'ejemplo1.json')
ejemplo2 = os.path.join(path_data, 'ejemplo2.json')
ejemplo3 = os.path.join(path_data, 'ejemplo3.json')

#### Ejemplo 1

In [9]:
pd.read_json(ejemplo1)

,nombre,productos
0,Tienda de Electrónica,"{'nombre_prod': 'Laptop', 'precio': 800, 'marc..."
1,Tienda de Electrónica,"{'nombre_prod': 'Teléfono', 'precio': 500, 'ma..."


#### json_normalize

**¿Qué hace `pd.json_normalize`?**

En esencia, esta función de Pandas toma datos JSON potencialmente complejos y anidados, y los transforma en un DataFrame plano y estructurado, que es mucho más fácil de manipular y analizar.

**Desglose de los parámetros:**

* `data`: Este es el JSON que contiene tus datos.

* `record_path='productos'`: Aquí le estás diciendo a Pandas: "Dentro de mi diccionario `data`, busca una lista llamada 'productos'. Cada elemento dentro de esa lista representa una fila en mi DataFrame final".

* `meta=['nombre']`:  Con esto indicas: "Además de los datos dentro de 'productos', quiero incluir una columna adicional llamada 'nombre' en mi DataFrame. El valor de esta columna será el valor asociado a la clave 'nombre' en el nivel superior de mi diccionario `data`".  En nuestro ejemplo, esto significa que cada fila tendrá el nombre de la tienda ("Tienda de Electrónica") junto con los detalles del producto.


In [10]:
data = json.load(open(ejemplo1))
pd.json_normalize(
    data,
    record_path='productos',
    meta=['nombre']
)

,nombre_prod,precio,marca,nombre
0,Laptop,800,Marca A,Tienda de Electrónica
1,Teléfono,500,Marca B,Tienda de Electrónica


#### Ejemplo 2

In [11]:
pd.read_json(ejemplo2)

,usuario
nombre,Juan
edad,30
direcciones,"[{'calle': 'Calle Principal 123', 'ciudad': 'C..."
compras,"[{'producto': 'Libro', 'precio': 20, 'fecha': ..."


In [12]:
data2 = json.load(open(ejemplo2))
pd.json_normalize(
    data2
)

,usuario.nombre,usuario.edad,usuario.direcciones,usuario.compras
0,Juan,30,"[{'calle': 'Calle Principal 123', 'ciudad': 'C...","[{'producto': 'Libro', 'precio': 20, 'fecha': ..."


In [13]:
pd.json_normalize(
    data2['usuario'],
    record_path=['direcciones'],
    meta=['nombre', 'edad',]
)

,calle,ciudad,id,nombre,edad
0,Calle Principal 123,Ciudad A,1,Juan,30
1,Avenida Secundaria 456,Ciudad B,2,Juan,30


In [14]:
pd.json_normalize(
        data2['usuario'],
    record_path=['compras'],
    meta=['nombre', 'edad',]
)

,producto,precio,fecha,id,nombre,edad
0,Libro,20,2020-01-01,1,Juan,30
1,Película,15,2020-01-02,2,Juan,30


#### Ejemplo 3

In [15]:
pd.read_json(ejemplo3)

,escuela
nombre,Escuela Primaria
grados,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [16]:
data3 = json.load(open(ejemplo3))
pd.json_normalize(
    data3
)

,escuela.nombre,escuela.grados
0,Escuela Primaria,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [17]:
pd.json_normalize(
    data3['escuela']['grados'],
    record_path='alumnos',
    meta=['nombre'],
    meta_prefix='grado_'
)

,nombre,edad,grado_nombre
0,Ana,6,Primer Grado
1,Pedro,7,Primer Grado
2,María,7,Segundo Grado
3,Luis,8,Segundo Grado
